In [ ]:
from ase.io import read
from ase import Atoms
from ase.optimize import *
from ase.visualize import view
from ase.md import *
from ase.calculators.mopac import  *
from ase.constraints import *
from math import *
import numpy as np
import nglview
import scipy.misc
import matplotlib.pyplot as pyplot
from ase.optimize.sciopt import SciPyFminBFGS, SciPyFminCG

In [ ]:


# In theis example 02.02_scan1D_diss we investigate the potential energy along the distance between 
# -CH3-Br. More precisely, we investigate dissociative type of mechanism for this reaction which in there first Br-
# -leaves the CH3 then Cl will attack to the remained CH3. 

In [ ]:
# Input intial file
model = "A"
infile  = "input/%s.xyz" % model
A   = read(infile, format="xyz")
molecule = Atoms(A)

# visualize initial structure
wdg_init = nglview.show_ase(molecule)
wdg_init.add_representation('ball+stick')
wdg_init.center_view(range(3))

In [ ]:
calc=Mopac(restart=0, spin=0, OPT=False, functional='PM6', job_type='NOANCI CHARGE=-1 NOSYM 1SCF GRADIENTS AUX(0,PRECISION=9)', RELSCF=0.0001 )

molecule.set_calculator(calc)

ener=molecule.get_potential_energy()
print "potential energy:", ener


atomC=0
atomA=4
atomB=5

bond1 = [molecule.get_distance(atomC, atomA), [atomC, atomA]]
bond2 = [molecule.get_distance(atomC, atomB), [atomC, atomB]]

print bond1, bond2

B1=molecule.get_distance(atomC, atomA)
B2=molecule.get_distance(atomC, atomB)

stepsize=0.1
numstep=10
print "stepping distance by (", stepsize, " Angstroms) "

scan=np.zeros(numstep)
d=np.zeros(numstep)
df=np.zeros(numstep)

for i in range (int(numstep)):

        d[i]=B2+i*stepsize
	print "step", i, "dist", d[i]
        molecule.set_distance(atomC, atomB, d[i], fix=0)
	outfile="output/" + "sn1.cons.drive."+str(i)+".xyz"
	molecule.write(outfile)

	bond1 = [d[i], [atomC, atomB]]
	cons = FixInternals(molecule,  bonds=[bond1])
	molecule.set_constraint(cons)


	ener=molecule.get_potential_energy()
	print "potential energy:", ener
	grad=molecule.get_forces()
	print "gradient", grad

	trajName="output/" + "drive.diss."+str(i)+".traj"
	dyn = QuasiNewton(molecule, trajectory=trajName)
	dyn.run(fmax=0.01, steps=10)

        df[i]=molecule.get_distance(atomC, atomB)
	print "dist after opt", df[i]
        ener=molecule.get_potential_energy()
	scan[i]=ener
	outfile_opt="output/" + "drive.diss."+str(i)+".opt.xyz"
	molecule.write(outfile_opt)



In [ ]:
# Plotting energy vs distance between cl and CH3-Br. Can you see the transition state?

pyplot.title('scan distance')
pyplot.ylabel('energy',fontsize=18)                
pyplot.xlabel('distance/Angstrom',fontsize=18)   
pyplot.plot(df,scan,'o',label="scan")
pyplot.show()


In [ ]:
# Compare the initial PDB files before and after Optimization - what we can learn?
wdg_init.display(gui=True)

In [ ]:
wdg = nglview.show_ase(molecule)
wdg.add_representation('ball+stick')
wdg.center_view(range(3))
wdg.display(gui=True)